In [2]:
import multiprocessing
import gensim, warnings
import re  # For preprocessing
import pandas as pd  # For data handling
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.phrases import Phrases, Phraser
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

warnings.filterwarnings(action="ignore")

In [3]:
#zamiana tekstu na listę
def Convert(string): 
    li = list(string.split("\n"))
    return li
def Convertcoma(string):
    li = list(string.split(","))
    return li
def ConvNEXT(string):
    li = list(string.split("NA5T3PNY"))
    return li

In [4]:
# Reads files
file = open("TEKST.txt", "r", encoding="utf-8")
# zródło: https://pl.wikipedia.org/wiki/Wikipedia:Stopwords
stopwords = open("stopwordsWIKI.txt", "r", encoding="utf8")
searched = open("WYSZUKANIA.txt", "r", encoding="utf-8")

In [5]:
# string = file.read()
stop = stopwords.read().lower()
stop=re.sub('[\s]+', '', stop) # usuwanie spacji z listy
# print(stop)
# Replaces escape character with space
stopwords_list = Convertcoma(stop)
# print(stopwords_list)
# print(len(stopwords_list))
wysz = searched.read().lower()
szukane = Convert(wysz)

In [6]:
string_org = file.read() #wczytanie pliku zawierającego wszystkie artykuły
print(len(string_org.split(' ')))

1960452


In [7]:
# zamknięcie otwartych plików
file.close()
stopwords.close()
searched.close()

In [8]:
# usunięcie wszystkich elementów tekstu np.'== Pliki zewnętrzne ==' które kopiowane z wikipedii są najo nagłówki kolejnych akapitów

# *     - 0 or more
# +     - 1 or more
# ?     - 0 or One
# {3}   - exact number
# {3,4} - range of numbers min max

string=re.sub('==[\s*\w*\s*]*==',' ',string_org) 
print(len(string.split(' '))) #wyświetlenie ile jest wyrazów

1930115


In [9]:
# usunięcie wszystkiego poza wyrazami i liczbami - uwuwanie znaków specjalnych :;,.*=+-/
string=re.sub('[^\w]+', ' ', string) 
print(len(string.split(' '))) #wyświetlenie ile jest wyrazów
# tutaj po usunięciu powstały nadmierne spacje które liczone są jako oddnielny ciąg znaków

1940042


In [10]:
# usunięcie wszystkich pojedynczych i podwójnych znaków np. r gdy coś było oznaczone jako rok 
# lub skróty tupu m.in. w.-wiek
string=' '.join( [w for w in string.split() if len(w)>2]) 
print(len(string.split(' '))) #wyświetlenie ile jest wyrazów

1546285


In [11]:
#Rozdzielenie artykułów z jednego długiego tekstu na każdy z osobna i zapisanie jako element listy
#Użyto do tego ciągu znaków 'NA5T3PNY' który był dodawany podczas zapisu na końcu każdego artykułu
tokeX=ConvNEXT(string)
print(tokeX[6])

Cyclops cunningtoni gatunek widłonoga rodziny Cyclopidae nazwa naukowa gatunku została raz pierwszy opublikowana 1909 roku przez norweskiego hydrobiologa Georga Ossiana Sarsa Walter Chad 2012 Cyclops cunningtoni Sars 1909 Walter Boxshall 2012 World Copepoda database Geraadpleegd via World Register Marine Species 


In [12]:
tokeXS=[]
q=0
for s in range(len(tokeX)):
    k=tokeX[s]
    k=k.lower() #małe litery
    # usunięcie wszystkich słów często występujących w tekście (lista słów z wikipedii)
    k=' '.join( [w for w in k.split() if w not in stopwords_list])
    k=re.sub('[\d]+', '',k) #usuwanie liczb
#     k=re.sub('[\s]+', ' ',k) #usuwanie nadmiernych spacji
    k=re.sub('[^\w]+', ' ',k) #usunięcie wszystkiego poza wyrazami - pozbycie sie nadmiernych spacji
    tokeXS.append(k) 
    a=len(k.split())
    q+=a
print(len(tokeXS))
print(q) #wyświetlenie ile jest wyrazów
    

4865
1280748


In [13]:
# wrzucenie wszystkich artykułów po oczyszczeniu do data frame
df = pd.DataFrame(tokeXS, columns =['Tekst'])
print(df)
df.shape

Tekst
0     kielisznik zaroślowy calystegia sepium gatunek...
1     model ekv tranzystora mos obliczeniowych model...
2      matsuobasho planetoida pasa głównego asteroid...
3     porwana historia carliny white ang abducted th...
4     zespół szkół poligraficznych marszałka józefa ...
...                                                 ...
4860  santa lucía reforma niewielkie miasto południo...
4861  piasek mały wieś polsce położona województwie ...
4862  gsm ang global system for mobile communication...
4863  teoria budżetu dyscyplina naukowa zajmująca po...
4864                                                   

[4865 rows x 1 columns]


(4865, 1)

In [14]:
df.isnull().sum() #puste wartości

Tekst    0
dtype: int64

In [15]:
print(df['Tekst'])

0       kielisznik zaroślowy calystegia sepium gatunek...
1       model ekv tranzystora mos obliczeniowych model...
2        matsuobasho planetoida pasa głównego asteroid...
3       porwana historia carliny white ang abducted th...
4       zespół szkół poligraficznych marszałka józefa ...
                              ...                        
4860    santa lucía reforma niewielkie miasto południo...
4861    piasek mały wieś polsce położona województwie ...
4862    gsm ang global system for mobile communication...
4863    teoria budżetu dyscyplina naukowa zajmująca po...
4864                                                     
Name: Tekst, Length: 4865, dtype: object


In [16]:
# podział każdego artykułu na pojedyncze słowa 
sent = [row.split() for row in df['Tekst']]

In [17]:
print(type(sent))
print(sent[7])

<class 'list'>
['densmore', 'den', 'ronald', 'dover', 'kwietnia', 'warrington', 'brytyjski', 'polityk', 'poseł', 'izby', 'gmin', 'deputowany', 'parlamentu', 'europejskiego', 'ukończył', 'studia', 'inżynierskie', 'university', 'manchester', 'pracował', 'przedsiębiorstwach', 'konstrukcyjnych', 'dyrektorem', 'krajowej', 'agencji', 'budownictwa', 'zasiadał', 'radzie', 'dzielnicy', 'barnet', 'londynie', 'cztery', 'kadencje', 'deputowanym', 'izby', 'gmin', 'latach', 'ramienia', 'partii', 'konserwatywnej', 'sprawował', 'mandat', 'posła', 'parlamentu', 'europejskiego', 'należał', 'frakcji', 'chadeckiej', 'pracował', 'komisji', 'budżetowej', 'komisji', 'przemysłu', 'badań', 'naukowych', 'energii', 'den', 'dover', 'przewodniczącym', 'torysów', 'europarlamencie', 'zrezygnował', 'ujawnieniu', 'dziewięć', 'wypłacił', 'publicznych', 'funduszy', 'tys', 'funtów', 'swojej', 'żonie', 'córce', 'wyniku', 'wewnętrznego', 'śledztwa', 'nakazano', 'zwrócić', 'sumy', 'usunięty', 'partii', 'konserwatywnej']


In [18]:
phrases = Phrases(sent, min_count=20, progress_per=10000)

INFO - 13:43:27: collecting all words and their counts
INFO - 13:43:27: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 13:43:29: collected 1125923 word types from a corpus of 1280748 words (unigram + bigrams) and 4865 sentences
INFO - 13:43:29: using 1125923 counts as vocab in Phrases<0 vocab, min_count=20, threshold=10.0, max_vocab_size=40000000>


In [19]:
bigram = Phraser(phrases)

INFO - 13:43:29: source_vocab length 1125923
INFO - 13:43:38: Phraser built with 1280 phrasegrams


In [20]:
sentences = bigram[sent]

##### Sprawdzenie efektywności oczyszczania tektu poprzez sprawdzenie jakie słowa występują najczęściej w pliku

In [21]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq) #ile jest różnych unikalnych słów

187632

##### Wyświetlamy 10 najczęściej występujących słów globalnie

In [22]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['latach',
 'polski',
 'the',
 'ang',
 'została',
 'miasta',
 'stycznia',
 'maja',
 'polsce',
 'września']

In [23]:
cores = multiprocessing.cpu_count() # Liczba obsługiwanych wątków przez procesor (u mnie 12)
print(cores)

12


##### Nadajemy parametry dla modelu word2vec (w2v_model)

In [24]:
w2v_model = Word2Vec(min_count=30, #ignoruje wszystkie wyrazy które wystąpiły żadziej niż-(2,100)
                     window=5, #max dystans pomiedzy słowem rozpatrywanym a przewidywanym w zdaniu-(2,10)
                     size=300, #wymiar wektorów - (50,300)
                     sample=6e-5, #próg dla którego bardzo często występujące słowa będą rzadziej brane pod uwagę (Duży wpływ)-(0,1e-5)
                     alpha=0.03, # początkowy wsp. uczenia - (0.01,0.05)
                     workers=cores-1) # ile wątków procesora wykorzystujemy

# UWAGA
tutaj można pominąć etap budowy słownika i przejść do wytrenowanego już modelu

Parametry wytrenowanego modelu:

min_count=30, #ignoruje wszystkie wyrazy które wystąpiły żadziej niż-(2,100)

window=5, #max dystans pomiedzy słowem rozpatrywanym a przewidywanym w zdaniu-(2,10)

size=300, #wymiar wektorów - (50,300)

sample=6e-5, #próg dla którego bardzo często występujące słowa będą rzadziej brane pod uwagę (Duży wpływ)-(0,1e-5)

alpha=0.03, # początkowy wsp. uczenia - (0.01,0.05)

workers=cores-1 # ile wątków procesora wykorzystujemy

### Nazwa pliku: w2v_model.bin

Tworzymy słownik 

In [25]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 21:48:45: collecting all words and their counts
INFO - 21:48:45: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 21:48:48: collected 187632 word types from a corpus of 1232606 raw words and 4865 sentences
INFO - 21:48:48: Loading a fresh vocabulary
INFO - 21:48:48: effective_min_count=30 retains 7161 unique words (3% of original 187632, drops 180471)
INFO - 21:48:48: effective_min_count=30 leaves 643936 word corpus (52% of original 1232606, drops 588670)
INFO - 21:48:48: deleting the raw counts dictionary of 187632 items
INFO - 21:48:48: sample=6e-05 downsamples 1554 most-common words
INFO - 21:48:48: downsampling leaves estimated 487053 word corpus (75.6% of prior 643936)
INFO - 21:48:48: estimated required memory for 7161 words and 300 dimensions: 20766900 bytes
INFO - 21:48:48: resetting layer weights
Time to build vocab: 0.08 mins


In [26]:
w2v_model.wv.vocab #podgląd na słowa występujące w słowniku

ctors.Vocab at 0x17e92037ec8>,
 'leży': <gensim.models.keyedvectors.Vocab at 0x17e92037048>,
 'dawnej': <gensim.models.keyedvectors.Vocab at 0x17e920370c8>,
 'krakowskiej': <gensim.models.keyedvectors.Vocab at 0x17e92037148>,
 'historycznej': <gensim.models.keyedvectors.Vocab at 0x17e920371c8>,
 'danych': <gensim.models.keyedvectors.Vocab at 0x17e92037208>,
 'miało': <gensim.models.keyedvectors.Vocab at 0x17e92037248>,
 'mieszkańców': <gensim.models.keyedvectors.Vocab at 0x17e92037288>,
 'dukli': <gensim.models.keyedvectors.Vocab at 0x17e920372c8>,
 'mieści': <gensim.models.keyedvectors.Vocab at 0x17e92037388>,
 'policji': <gensim.models.keyedvectors.Vocab at 0x17e920373c8>,
 'ośrodek': <gensim.models.keyedvectors.Vocab at 0x17e92037408>,
 'zdrowia': <gensim.models.keyedvectors.Vocab at 0x17e92037448>,
 'dom': <gensim.models.keyedvectors.Vocab at 0x17e920374c8>,
 'pttk': <gensim.models.keyedvectors.Vocab at 0x17e92037508>,
 'ratusz': <gensim.models.keyedvectors.Vocab at 0x17e920375c8>,

Trenowanie modelu gdzie:

total_examples = całkowita liczba wszystkich zdań (w naszym przypadku liczba wszystkich artykułów);

epochs = całkowita liczba iteracji (epochs) przejścia przez corpus - (10, 20, 30)


In [27]:
print(w2v_model.corpus_count)

4865


In [28]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

SS: at 11.59% examples, 57163 words/s, in_qsize 22, out_qsize 0
INFO - 21:49:57: EPOCH 15 - PROGRESS: at 45.34% examples, 105372 words/s, in_qsize 22, out_qsize 2
INFO - 21:49:58: EPOCH 15 - PROGRESS: at 83.80% examples, 126771 words/s, in_qsize 22, out_qsize 0
INFO - 21:49:58: worker thread finished; awaiting finish of 10 more threads
INFO - 21:49:58: worker thread finished; awaiting finish of 9 more threads
INFO - 21:49:58: worker thread finished; awaiting finish of 8 more threads
INFO - 21:49:58: worker thread finished; awaiting finish of 7 more threads
INFO - 21:49:58: worker thread finished; awaiting finish of 6 more threads
INFO - 21:49:58: worker thread finished; awaiting finish of 5 more threads
INFO - 21:49:58: worker thread finished; awaiting finish of 4 more threads
INFO - 21:49:58: worker thread finished; awaiting finish of 3 more threads
INFO - 21:49:58: worker thread finished; awaiting finish of 2 more threads
INFO - 21:49:58: worker thread finished; awaiting finish of 1 

##### Zapis wytrenowanego modelu do pliku

In [25]:
w2v_model.wv.save_word2vec_format('w2v_model.bin', binary=True)

RuntimeError: no input

# Wczytanie zapisanego modelu
Wczytanie zapisanego wcześniej modelu z pliku

## Jeśli wytrenowaliśmy model na nowych parametrach należy pominąć poniższą komórkę 

In [26]:
w2v_model = KeyedVectors.load_word2vec_format("w2v_model.bin", binary=True)

INFO - 13:44:24: loading projection weights from w2v_model.bin
INFO - 13:44:25: loaded (7161, 300) matrix from w2v_model.bin


In [27]:
w2v_model.wv.vocab #podgląd na słowa występujące w słowniku
print(1)

1


#### ---------------------------------------------------------------------------

In [28]:
w2v_model.wv.most_similar(positive=["zamek"])

INFO - 13:44:37: precomputing L2-norms of word weight vectors


[('pałac', 0.786061704158783),
 ('zamku', 0.7524723410606384),
 ('królewski', 0.6971192359924316),
 ('książąt', 0.642208456993103),
 ('ruiny', 0.6317052245140076),
 ('gród', 0.6049981117248535),
 ('ratusz', 0.5893390774726868),
 ('murowany', 0.5868734121322632),
 ('królewskie', 0.5799057483673096),
 ('wzniesiono', 0.5479350090026855)]

In [34]:
w2v_model.wv.most_similar(positive=["święto"])

[('święta', 0.7786576747894287),
 ('uroczystości', 0.7060004472732544),
 ('patrona', 0.6873683929443359),
 ('miłosierdzia_bożego', 0.6813386678695679),
 ('bożego', 0.6794769763946533),
 ('grobie', 0.6775121688842773),
 ('uroczystość', 0.6705838441848755),
 ('rocznicy', 0.6526843905448914),
 ('czci', 0.6455272436141968),
 ('obchodów', 0.6427680253982544)]

In [35]:
w2v_model.wv.most_similar(positive=["liceum"])

[('gimnazjum', 0.8220643401145935),
 ('technikum', 0.819947361946106),
 ('liceum_ogólnokształcące', 0.8157960176467896),
 ('szkoły', 0.7762669324874878),
 ('szkole', 0.7579822540283203),
 ('zespół_szkół', 0.7516874074935913),
 ('naukę', 0.7291847467422485),
 ('szkoła', 0.7274847030639648),
 ('szkoły_podstawowej', 0.7118510007858276),
 ('stanisława_staszica', 0.7067950367927551)]

In [36]:
w2v_model.wv.most_similar(positive=["mistrz"])

[('dwukrotny', 0.7735169529914856),
 ('wicemistrz', 0.7612346410751343),
 ('brązowy_medalista', 0.7119187712669373),
 ('finalista', 0.6522637605667114),
 ('medalista_olimpijski', 0.6406358480453491),
 ('lekkoatleta', 0.638056218624115),
 ('uczestnik', 0.6361618638038635),
 ('mistrz_olimpijski', 0.6016188859939575),
 ('mistrzem', 0.5968785285949707),
 ('mistrzyni', 0.5881224274635315)]

In [37]:
w2v_model.wv.most_similar(positive=["król"])

[('książę', 0.7753475904464722),
 ('królem', 0.742949366569519),
 ('fryderyk', 0.7417298555374146),
 ('władca', 0.7406895160675049),
 ('karol', 0.7358067035675049),
 ('panowanie', 0.7193434238433838),
 ('august', 0.6869291663169861),
 ('tron', 0.6838666200637817),
 ('habsburg', 0.681429386138916),
 ('dynastii', 0.6802775859832764)]

In [38]:
w2v_model.wv.most_similar(positive=["królowa"])

[('elżbieta', 0.830079197883606),
 ('królową', 0.799121081829071),
 ('królowej', 0.7824377417564392),
 ('elżbiety', 0.7071044445037842),
 ('szkocji', 0.5908934473991394),
 ('queen', 0.5388177037239075),
 ('królewska', 0.5307820439338684),
 ('walii', 0.5283026695251465),
 ('tronu', 0.527069091796875),
 ('mężem', 0.5260130763053894)]

In [39]:
w2v_model.wv.most_similar(positive=["kolor"]) 

[('żółty', 0.8178884983062744),
 ('niebieski', 0.8173955678939819),
 ('czarny', 0.7851782441139221),
 ('białym', 0.7674051523208618),
 ('czarnym', 0.7511237859725952),
 ('kolorze', 0.7400006055831909),
 ('zielony', 0.7337899208068848),
 ('koloru', 0.7334598898887634),
 ('czerwonym', 0.6677516102790833),
 ('czarne', 0.6507636308670044)]

In [40]:
w2v_model.wv.most_similar(positive=["auto"]) 

[('mazda', 0.8115215301513672),
 ('mustanga', 0.7985462546348572),
 ('samochodu', 0.7895808815956116),
 ('nadwozia', 0.7857412695884705),
 ('samochód', 0.761103093624115),
 ('auta', 0.75453782081604),
 ('mustang', 0.7527592182159424),
 ('modele', 0.747249960899353),
 ('zawieszenie', 0.7194379568099976),
 ('samochodem', 0.715705931186676)]

In [41]:
w2v_model.wv.most_similar(positive=["liczba"]) 

[('duża', 0.6930006146430969),
 ('wzrosła', 0.6708990335464478),
 ('zarejestrowanych', 0.6533222794532776),
 ('wynosiła', 0.6461988687515259),
 ('procent', 0.6381279826164246),
 ('wyniosła', 0.622017502784729),
 ('liczby', 0.6190260052680969),
 ('ilość', 0.6142896413803101),
 ('mężczyzn_kobiet', 0.610946536064148),
 ('łączna', 0.5991466045379639)]

In [42]:
w2v_model.wv.most_similar(positive=["liczby"]) 

[('zmniejszenie', 0.7458816170692444),
 ('największej', 0.7203786373138428),
 ('procent', 0.7080565094947815),
 ('danym', 0.6865778565406799),
 ('większej', 0.6767949461936951),
 ('liczbę', 0.6733611822128296),
 ('stałych', 0.6545504331588745),
 ('najmniej', 0.6334962844848633),
 ('odpowiednio', 0.6240201592445374),
 ('różnica', 0.6200425624847412)]

In [ ]:
w2v_model.wv.most_similar(positive=["pięć"]) 

In [ ]:
w2v_model.wv.most_similar(positive=["wzrost"]) 

In [ ]:
w2v_model.wv.most_similar(positive=["polska"]) 

In [ ]:
w2v_model.wv.most_similar(positive=["francja"]) 

In [ ]:
w2v_model.wv.most_similar(positive=["język"]) 

In [ ]:
w2v_model.wv.most_similar(positive=["złoto"]) 

In [ ]:
w2v_model.wv.most_similar(positive=["książka"]) 

In [ ]:
w2v_model.wv.most_similar(positive=["owoce"]) 

In [ ]:
w2v_model.wv.most_similar(positive=["noc"]) 

In [ ]:
w2v_model.wv.most_similar(positive=["kot"]) 

In [ ]:
w2v_model.wv.most_similar(positive=["program"]) 

In [ ]:
w2v_model.wv.most_similar(positive=["gra"]) 

In [ ]:
w2v_model.wv.most_similar(positive=["gry"]) 

In [ ]:
w2v_model.wv.most_similar(positive=["grę"]) 

Gdy już nie planujemy dalej trenować modelu, wywołujemy funkcję init_sims (), co sprawi, że model będzie znacznie bardziej wydajny pod względem pamięci:

In [ ]:
w2v_model.init_sims(replace=False)